In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = np.array(pd.read_csv('Churn_Modelling.csv'))
data = data
data.shape
np.random.seed(12)

In [3]:
X = data[:, 3:-1]
y = data[:, -1]
print(X)
print(y)
print((y == 0).sum())

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]
7963


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
encoder_1 = LabelEncoder()
X[:, 1] = encoder_1.fit_transform(X[:, 1])
encoder_2 = LabelEncoder()
X[:, 2] = encoder_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = X.astype(float)
y = y.astype(float)
print(X)
y = y.reshape(-1, 1)
number_of_training_examples = int(X.shape[0]*0.2)
X_test = X[int(X.shape[0]*0.8):, :]
X = X[:int(X.shape[0]*0.8), :]
y_test = y[int(y.shape[0]*0.8):, :]
y = y[:int(y.shape[0]*0.8), :]
print(str(X.shape) + " " + str(y.shape))

[[-0.57873591 -0.57380915 -0.32622142 ...  0.64609167  0.97024255
   0.02188649]
 [-0.57873591  1.74273971 -0.44003595 ... -1.54776799  0.97024255
   0.21653375]
 [-0.57873591 -0.57380915 -1.53679418 ...  0.64609167 -1.03067011
   0.2406869 ]
 ...
 [-0.57873591 -0.57380915  0.60498839 ... -1.54776799  0.97024255
  -1.00864308]
 [ 1.72790383 -0.57380915  1.25683526 ...  0.64609167 -1.03067011
  -0.12523071]
 [-0.57873591 -0.57380915  1.46377078 ...  0.64609167 -1.03067011
  -1.07636976]]
(8000, 11) (8000, 1)


In [5]:
learning_rate = 0.001
original = learning_rate
number_of_inputs = X.shape[0]
input_size = X.shape[1]
hidden_size = 8
output_size = 1
epochs = 1
number_of_inputs

8000

In [6]:
Wxh = np.random.randn(hidden_size, input_size).astype(float) * 0.01
Who = np.random.randn(output_size, hidden_size).astype(float) * 0.01
bh = np.zeros((hidden_size, 1)).astype(float)
bo = 0.0
print(Wxh)

[[ 4.72985831e-03 -6.81425879e-03  2.42439497e-03 -1.70073563e-02
   7.53142834e-03 -1.53472134e-02  5.12707813e-05 -1.20227670e-03
  -8.06981879e-03  2.87181939e-02 -5.97822919e-03]
 [ 4.72456994e-03  1.09595612e-02 -1.21516880e-02  1.34235637e-02
  -1.22149788e-03  1.01251548e-02 -9.13869147e-03 -1.02953021e-02
   1.20979645e-02  5.01872304e-03  1.38846177e-03]
 [ 6.40761113e-03  5.27332665e-03 -1.15436024e-02 -2.21333348e-02
  -1.68175651e-02 -1.78809425e-02 -2.21853495e-02 -6.47430780e-03
  -5.28404321e-03 -3.92091748e-04  2.14975948e-03]
 [-3.84358803e-03 -2.53904078e-03  7.32520746e-04 -9.97203837e-03
  -7.13856288e-03  3.54163461e-04 -6.77945365e-03 -5.71881062e-03
  -1.05862315e-03  1.33583134e-02  3.18665288e-03]
 [-3.37595250e-03 -5.85268278e-03 -1.14919941e-03  2.24181779e-02
  -3.14741652e-02  5.35135893e-03  2.32490439e-03  8.67611951e-03
  -1.14821271e-02  2.11434424e-02  1.00094276e-02]
 [-5.14149953e-04  1.59787698e-03 -7.16263586e-03  5.05228265e-04
  -1.43337415e-03  

In [7]:
def nonlin(x, derivative = False):
    if(derivative == True):
        return (x*(1-x))
    return 1 / (1 + np.exp(-x))

In [8]:
def fprop(inputs, y):
    l1 = nonlin(np.dot(Wxh, inputs))
    l2 = nonlin(np.dot(Who, l1))
    print(l2, y)
    return l1, l2

In [9]:
def bprop(l1, l2, y):
#     print((-(1 - y) * (np.log(1 - l2)) - (y * np.log(l2))), y, end = "\n")
    del_l2 = (y - l2) * nonlin(l2, True)
    del_l1 = (np.dot(Who.T, del_l2)) * nonlin(l1, True)
    return del_l1, del_l2

In [10]:
for i in range(5):
    for j in range(X.shape[0]):
        l1, l2 = fprop(X[j], y[j])
        del_l1, del_l2 = bprop(l1, l2, y[j])
        if(y[j] == 1):
            learning_rate = 1
        else:
            learning_rate = 0.5
#         print(learning_rate)
        Wxh = Wxh + learning_rate * np.dot(del_l1.reshape(-1, 1), X[j].reshape(-1, 1).T)
        Who = Who + learning_rate * np.dot(del_l2.reshape(-1, 1), l1.reshape(-1, 1).T)
        learning_rate = original
        

[0.49703455] [1.]
[0.56075604] [0.]
[0.52557686] [1.]
[0.58354106] [0.]
[0.54907098] [0.]
[0.51562963] [1.]
[0.57472483] [0.]
[0.54180502] [1.]
[0.59857653] [0.]
[0.56255564] [0.]
[0.52840293] [0.]
[0.49526699] [0.]
[0.46337481] [0.]
[0.43379455] [0.]
[0.40743002] [0.]
[0.38585682] [0.]
[0.36656285] [1.]
[0.42915749] [0.]
[0.40545722] [0.]
[0.3790877] [0.]
[0.35860585] [0.]
[0.3351106] [0.]
[0.32015252] [1.]
[0.39460309] [0.]
[0.37038777] [0.]
[0.34656911] [0.]
[0.33538699] [0.]
[0.31653536] [0.]
[0.30591553] [0.]
[0.28698732] [0.]
[0.27409114] [1.]
[0.33254255] [0.]
[0.31960758] [0.]
[0.31069239] [0.]
[0.29058442] [0.]
[0.29136955] [1.]
[0.33910133] [0.]
[0.31410974] [0.]
[0.30004553] [0.]
[0.28115932] [0.]
[0.29282326] [0.]
[0.27378986] [1.]
[0.34307207] [0.]
[0.32537204] [1.]
[0.38491922] [0.]
[0.35130952] [0.]
[0.32614035] [1.]
[0.41192611] [1.]
[0.4831138] [0.]
[0.4562241] [0.]
[0.42714421] [0.]
[0.402159] [0.]
[0.38072401] [0.]
[0.36077905] [1.]
[0.43119112] [1.]
[0.49253425] [0.

[0.23348221] [0.]
[0.24444396] [0.]
[0.19451288] [1.]
[0.19521165] [0.]
[0.27940063] [0.]
[0.18613851] [0.]
[0.27533089] [1.]
[0.27632806] [0.]
[0.21804628] [0.]
[0.23529245] [0.]
[0.32186482] [1.]
[0.31788344] [0.]
[0.24809379] [0.]
[0.28209173] [0.]
[0.3863294] [1.]
[0.32630287] [0.]
[0.32235481] [0.]
[0.22843574] [0.]
[0.21718053] [0.]
[0.24787789] [0.]
[0.19894919] [0.]
[0.18759966] [0.]
[0.28585247] [0.]
[0.3071061] [1.]
[0.26650152] [0.]
[0.23329173] [0.]
[0.24141292] [1.]
[0.22820097] [0.]
[0.39189192] [0.]
[0.25248753] [0.]
[0.29776894] [0.]
[0.21763817] [0.]
[0.31206449] [1.]
[0.34891061] [1.]
[0.3147502] [0.]
[0.35215624] [0.]
[0.27952642] [0.]
[0.23187916] [0.]
[0.3257444] [1.]
[0.28124095] [0.]
[0.30156561] [0.]
[0.41079022] [1.]
[0.31489074] [0.]
[0.39086007] [0.]
[0.27190034] [0.]
[0.33298373] [0.]
[0.38352811] [0.]
[0.16267672] [0.]
[0.21503192] [0.]
[0.1998998] [0.]
[0.30324413] [0.]
[0.22239553] [0.]
[0.18663984] [0.]
[0.29573877] [1.]
[0.23533084] [0.]
[0.241156] [1.]

[0.21894191] [0.]
[0.28893839] [0.]
[0.11493114] [0.]
[0.5899073] [1.]
[0.38244652] [1.]
[0.34092544] [0.]
[0.24865581] [0.]
[0.36424868] [0.]
[0.32908642] [0.]
[0.54377813] [0.]
[0.29630051] [0.]
[0.10707003] [0.]
[0.46594884] [0.]
[0.1897126] [0.]
[0.19647718] [0.]
[0.11869695] [0.]
[0.43094869] [1.]
[0.47966829] [0.]
[0.49423731] [0.]
[0.49850837] [0.]
[0.17877819] [0.]
[0.52090703] [0.]
[0.06488283] [0.]
[0.38925691] [1.]
[0.07248993] [0.]
[0.10035998] [0.]
[0.2886704] [0.]
[0.28451579] [1.]
[0.10501256] [0.]
[0.53257902] [1.]
[0.26249863] [0.]
[0.2278385] [0.]
[0.33746245] [0.]
[0.25227705] [0.]
[0.25962351] [0.]
[0.06810341] [0.]
[0.11203646] [0.]
[0.3973787] [0.]
[0.12014821] [0.]
[0.39483327] [0.]
[0.45952774] [0.]
[0.09463399] [0.]
[0.1460489] [0.]
[0.43443481] [1.]
[0.27364719] [0.]
[0.50577325] [1.]
[0.30768673] [0.]
[0.10794896] [0.]
[0.36879723] [1.]
[0.55732655] [0.]
[0.16955912] [1.]
[0.13978524] [0.]
[0.15342125] [0.]
[0.10554457] [0.]
[0.37432442] [0.]
[0.19293172] [1.

[0.15789997] [0.]
[0.25988841] [0.]
[0.717726] [1.]
[0.21946126] [0.]
[0.16725825] [0.]
[0.41594452] [0.]
[0.50944639] [0.]
[0.5309901] [0.]
[0.59375254] [1.]
[0.50811929] [1.]
[0.20286775] [0.]
[0.42547222] [0.]
[0.09704982] [0.]
[0.26404776] [0.]
[0.14484935] [0.]
[0.18891028] [0.]
[0.07407414] [0.]
[0.10920688] [0.]
[0.61278949] [1.]
[0.13823303] [0.]
[0.09883321] [0.]
[0.29000515] [0.]
[0.07918737] [0.]
[0.08317448] [0.]
[0.05963724] [0.]
[0.57513389] [1.]
[0.21522223] [0.]
[0.40679431] [1.]
[0.14581071] [1.]
[0.28033989] [0.]
[0.3623302] [0.]
[0.72979613] [1.]
[0.64177948] [1.]
[0.37366141] [0.]
[0.42819718] [0.]
[0.66430332] [0.]
[0.38426021] [0.]
[0.37214197] [0.]
[0.09953186] [0.]
[0.09126067] [0.]
[0.32662008] [1.]
[0.1457086] [0.]
[0.54740029] [0.]
[0.6535436] [1.]
[0.19049955] [0.]
[0.74414802] [1.]
[0.17012684] [0.]
[0.75604485] [0.]
[0.10444098] [0.]
[0.40299242] [0.]
[0.10225455] [0.]
[0.4681444] [0.]
[0.08923259] [0.]
[0.44868272] [0.]
[0.08912514] [0.]
[0.13807608] [0.]

[0.39436319] [0.]
[0.63288866] [0.]
[0.2921948] [0.]
[0.67418989] [1.]
[0.47605448] [0.]
[0.61805849] [1.]
[0.48154859] [1.]
[0.77347119] [0.]
[0.08345664] [0.]
[0.74350103] [0.]
[0.14761668] [0.]
[0.70095166] [0.]
[0.10070856] [0.]
[0.47752743] [0.]
[0.40247818] [0.]
[0.43634611] [0.]
[0.2257787] [0.]
[0.40727053] [0.]
[0.08586886] [1.]
[0.52932259] [0.]
[0.06041851] [0.]
[0.05899293] [0.]
[0.67431534] [0.]
[0.21840044] [0.]
[0.31115617] [0.]
[0.33469451] [0.]
[0.5504363] [1.]
[0.66960103] [0.]
[0.34477947] [1.]
[0.62995144] [1.]
[0.06674023] [0.]
[0.10587371] [0.]
[0.15915657] [0.]
[0.09573871] [0.]
[0.7279084] [1.]
[0.64703917] [1.]
[0.29285294] [0.]
[0.71054007] [1.]
[0.14425744] [0.]
[0.24896372] [0.]
[0.4812493] [0.]
[0.48723709] [0.]
[0.04913875] [0.]
[0.62700332] [0.]
[0.23967716] [0.]
[0.11830064] [0.]
[0.13567983] [1.]
[0.16967203] [0.]
[0.62118926] [1.]
[0.4245022] [0.]
[0.70592879] [1.]
[0.53532261] [0.]
[0.09596091] [0.]
[0.72137346] [0.]
[0.37115911] [0.]
[0.23585827] [0.

...

[0.74380609] [1.]
[0.31547798] [0.]
[0.4475263] [0.]
[0.48112013] [1.]
[0.83330844] [1.]
[0.41246978] [0.]
[0.00765492] [0.]
[0.00823505] [0.]
[0.91463196] [1.]
[0.29218523] [0.]
[0.12288017] [1.]
[0.08217533] [0.]
[0.08781856] [0.]
[0.0084844] [1.]
[0.1059925] [0.]
[0.01984161] [0.]
[0.00948005] [0.]
[0.09759101] [1.]
[0.37607102] [0.]
[0.97659497] [1.]
[0.17481457] [0.]
[0.42802497] [0.]
[0.07673941] [0.]
[0.03883664] [0.]
[0.3070391] [0.]
[0.63940699] [0.]
[0.55412126] [1.]
[0.9169458] [1.]
[0.0253935] [0.]
[0.00584593] [1.]
[0.15398374] [0.]
[0.00880532] [0.]
[0.10110404] [1.]
[0.73203881] [0.]
[0.12501638] [0.]
[0.19916075] [0.]
[0.09273177] [0.]
[0.60512767] [1.]
[0.00970777] [0.]
[0.42915054] [1.]
[0.65318014] [0.]
[0.12394725] [0.]
[0.05097871] [0.]
[0.81067395] [0.]
[0.16032692] [0.]
[0.00854132] [0.]
[0.00488195] [0.]
[0.08380807] [0.]
[0.05847046] [0.]
[0.96693308] [1.]
[0.08297459] [0.]
[0.12879674] [0.]
[0.0337537] [0.]
[0.49968703] [1.]
[0.16138882] [0.]
[0.30190277] [0.]

[0.21052089] [0.]
[0.050832] [1.]
[0.13754901] [0.]
[0.35712426] [0.]
[0.03990336] [0.]
[0.5407986] [1.]
[0.09099242] [0.]
[0.31801405] [0.]
[0.22080506] [0.]
[0.06117402] [0.]
[0.43591464] [0.]
[0.22071961] [0.]
[0.07216738] [0.]
[0.22475549] [0.]
[0.20929226] [0.]
[0.58939541] [0.]
[0.07218861] [1.]
[0.61477502] [1.]
[0.43620696] [0.]
[0.07164206] [0.]
[0.06909151] [0.]
[0.5802455] [0.]
[0.02623214] [0.]
[0.05851154] [0.]
[0.03168023] [0.]
[0.12512077] [0.]
[0.43279643] [1.]
[0.13442234] [0.]
[0.03108115] [0.]
[0.10678369] [0.]
[0.63235184] [0.]
[0.06845278] [0.]
[0.01427393] [0.]
[0.02393632] [0.]
[0.55349534] [0.]
[0.04747645] [0.]
[0.52037301] [0.]
[0.01890061] [0.]
[0.1235045] [0.]
[0.07537877] [0.]
[0.00530384] [0.]
[0.19906475] [1.]
[0.00186091] [0.]
[0.28395413] [1.]
[0.0418521] [1.]
[0.13349889] [0.]
[0.01997648] [0.]
[0.71327992] [0.]
[0.09238578] [0.]
[0.67193619] [1.]
[0.01755355] [0.]
[0.05493421] [0.]
[0.5214227] [1.]
[0.07846154] [0.]
[0.10660163] [0.]
[0.1227678] [0.]


[0.13615491] [0.]
[0.01909563] [0.]
[0.08116741] [0.]
[0.06987024] [0.]
[0.04325296] [0.]
[0.47152686] [0.]
[0.68569054] [1.]
[0.38397463] [1.]
[0.18199309] [0.]
[0.43937682] [0.]
[0.18086713] [0.]
[0.36755558] [0.]
[0.06126645] [0.]
[0.02431238] [0.]
[0.04720603] [0.]
[0.11911823] [0.]
[0.04338024] [0.]
[0.18134633] [0.]
[0.07281579] [0.]
[0.28406618] [0.]
[0.4281295] [0.]
[0.06997352] [0.]
[0.21281598] [0.]
[0.38840678] [0.]
[0.03959603] [0.]
[0.00988254] [0.]
[0.03585362] [1.]
[0.05445464] [0.]
[0.13352803] [0.]
[0.13485877] [1.]
[0.24740147] [0.]
[0.36159991] [0.]
[0.20076034] [0.]
[0.05519889] [0.]
[0.65239771] [0.]
[0.49745541] [0.]
[0.02881387] [0.]
[0.28793422] [0.]
[0.1129362] [0.]
[0.06005813] [0.]
[0.18946846] [0.]
[0.0285774] [0.]
[0.84253635] [1.]
[0.00927956] [0.]
[0.00114779] [0.]
[0.01981752] [0.]
[0.78876897] [0.]
[0.08067503] [0.]
[0.44755587] [0.]
[0.48032139] [0.]
[0.22715034] [1.]
[0.10158219] [0.]
[0.07337509] [0.]
[0.03030228] [0.]
[0.11452122] [0.]
[0.04016401] 

[0.01001992] [0.]
[0.03430335] [0.]
[0.33354706] [1.]
[0.5414432] [1.]
[0.84502503] [1.]
[0.26657058] [0.]
[0.07742699] [0.]
[0.62309935] [0.]
[0.20240636] [0.]
[0.06859808] [0.]
[0.01830435] [0.]
[0.53883046] [1.]
[0.06342226] [0.]
[0.88622385] [1.]
[0.94038599] [1.]
[0.0408375] [0.]
[0.24784589] [0.]
[0.14994447] [0.]
[0.81536474] [0.]
[0.07167622] [0.]
[0.05779621] [0.]
[0.21822988] [0.]
[0.01710797] [0.]
[0.79315043] [1.]
[0.06363455] [0.]
[0.0218262] [0.]
[0.15698152] [1.]
[0.1229757] [0.]
[0.15090057] [0.]
[0.04499212] [0.]
[0.15124095] [0.]
[0.58798018] [1.]
[0.08515332] [0.]
[0.50634672] [1.]
[0.10501455] [0.]
[0.89287571] [1.]
[0.7676281] [0.]
[0.017739] [0.]
[0.11516307] [0.]
[0.03911629] [0.]
[0.59784004] [0.]
[0.45411298] [0.]
[0.63104394] [1.]
[0.46293955] [0.]
[0.46536273] [0.]
[0.20521006] [0.]
[0.00872105] [0.]
[0.31959833] [0.]
[0.37291033] [0.]
[0.90737796] [0.]
[0.41341458] [0.]
[0.05798174] [0.]
[0.7513671] [1.]
[0.33155136] [0.]
[0.51168622] [0.]
[0.93257076] [1.]


[0.05690052] [0.]
[0.09978641] [0.]
[0.0463406] [0.]
[0.03928985] [0.]
[0.04392007] [0.]
[0.90753183] [1.]
[0.49645085] [1.]
[0.24280723] [0.]
[0.22022624] [0.]
[0.1384285] [0.]
[0.62116903] [0.]
[0.16831123] [0.]
[0.15181216] [0.]
[0.0207343] [0.]
[0.01448757] [0.]
[0.01542551] [0.]
[0.26305257] [1.]
[0.62522048] [0.]
[0.04421317] [0.]
[0.29220003] [0.]
[0.19199918] [0.]
[0.50264363] [1.]
[0.41682722] [1.]
[0.21962624] [1.]
[0.08031877] [0.]
[0.74985104] [0.]
[0.38418807] [0.]
[0.03142263] [0.]
[0.94845469] [1.]
[0.46846675] [0.]
[0.20905229] [0.]
[0.0499956] [0.]
[0.1276008] [0.]
[0.19952385] [0.]
[0.47189517] [0.]
[0.06663326] [0.]
[0.02902258] [0.]
[0.02548157] [0.]
[0.2265737] [0.]
[0.09937698] [0.]
[0.1654072] [0.]
[0.87887829] [1.]
[0.02944082] [0.]
[0.2212641] [0.]
[0.91346268] [1.]
[0.04112671] [0.]
[0.20174536] [0.]
[0.15673009] [0.]
[0.94848751] [1.]
[0.01117272] [0.]
[0.27819533] [0.]
[0.0927171] [0.]
[0.55683666] [0.]
[0.54024049] [1.]
[0.09010831] [0.]
[0.10645405] [0.]
[

In [11]:
s = np.zeros(number_of_training_examples)
for i in range(number_of_training_examples):
    hidden, output = fprop(X_test[i].reshape(-1, 1), y_test[i])
    print(output)
    s[i] = output

[[0.02525975]] [0.]
[[0.02525975]]
[[0.02287932]] [0.]
[[0.02287932]]
[[0.04438919]] [0.]
[[0.04438919]]
[[0.34603028]] [1.]
[[0.34603028]]
[[0.03076377]] [0.]
[[0.03076377]]
[[0.22339999]] [0.]
[[0.22339999]]
[[0.05793799]] [0.]
[[0.05793799]]
[[0.03640016]] [1.]
[[0.03640016]]
[[0.66037612]] [0.]
[[0.66037612]]
[[0.0498252]] [0.]
[[0.0498252]]
[[0.8150591]] [1.]
[[0.8150591]]
[[0.05664958]] [0.]
[[0.05664958]]
[[0.01885271]] [0.]
[[0.01885271]]
[[0.0595988]] [0.]
[[0.0595988]]
[[0.14170398]] [0.]
[[0.14170398]]
[[0.00751043]] [0.]
[[0.00751043]]
[[0.09585021]] [0.]
[[0.09585021]]
[[0.91570546]] [0.]
[[0.91570546]]
[[0.25715216]] [1.]
[[0.25715216]]
[[0.4127591]] [0.]
[[0.4127591]]
[[0.280554]] [0.]
[[0.280554]]
[[0.39235716]] [0.]
[[0.39235716]]
[[0.24335913]] [1.]
[[0.24335913]]
[[0.02180709]] [0.]
[[0.02180709]]
[[0.00517269]] [0.]
[[0.00517269]]
[[0.00310322]] [0.]
[[0.00310322]]
[[0.00928448]] [0.]
[[0.00928448]]
[[0.07860436]] [1.]
[[0.07860436]]
[[0.01583766]] [0.]
[[0.01583766

[[0.03548435]]
[[0.19292233]] [0.]
[[0.19292233]]
[[0.02940282]] [0.]
[[0.02940282]]
[[0.06526452]] [0.]
[[0.06526452]]
[[0.0860471]] [0.]
[[0.0860471]]
[[0.06651201]] [0.]
[[0.06651201]]
[[0.30269566]] [0.]
[[0.30269566]]
[[0.00292032]] [0.]
[[0.00292032]]
[[0.10537888]] [0.]
[[0.10537888]]
[[0.04495515]] [0.]
[[0.04495515]]
[[0.24133157]] [0.]
[[0.24133157]]
[[0.08729587]] [0.]
[[0.08729587]]
[[0.0501545]] [0.]
[[0.0501545]]
[[0.06475059]] [0.]
[[0.06475059]]
[[0.67487641]] [0.]
[[0.67487641]]
[[0.05668048]] [0.]
[[0.05668048]]
[[0.16092383]] [0.]
[[0.16092383]]
[[0.01982054]] [0.]
[[0.01982054]]
[[0.0598163]] [0.]
[[0.0598163]]
[[0.17772428]] [0.]
[[0.17772428]]
[[0.95271068]] [1.]
[[0.95271068]]
[[0.79237163]] [1.]
[[0.79237163]]
[[0.45934949]] [0.]
[[0.45934949]]
[[0.5150026]] [1.]
[[0.5150026]]
[[0.09241344]] [0.]
[[0.09241344]]
[[0.17431053]] [0.]
[[0.17431053]]
[[0.41624807]] [0.]
[[0.41624807]]
[[0.85509333]] [1.]
[[0.85509333]]
[[0.18795352]] [1.]
[[0.18795352]]
[[0.00692514]

[[0.03672353]]
[[0.02852371]] [0.]
[[0.02852371]]
[[0.37859967]] [0.]
[[0.37859967]]
[[0.38518641]] [0.]
[[0.38518641]]
[[0.30135935]] [0.]
[[0.30135935]]
[[0.01277447]] [0.]
[[0.01277447]]
[[0.034801]] [0.]
[[0.034801]]
[[0.32139344]] [0.]
[[0.32139344]]
[[0.04992072]] [0.]
[[0.04992072]]
[[0.02682065]] [0.]
[[0.02682065]]
[[0.02761775]] [0.]
[[0.02761775]]
[[0.1002658]] [0.]
[[0.1002658]]
[[0.50478533]] [0.]
[[0.50478533]]
[[0.26455996]] [0.]
[[0.26455996]]
[[0.04123107]] [1.]
[[0.04123107]]
[[0.19522273]] [1.]
[[0.19522273]]
[[0.1616289]] [0.]
[[0.1616289]]
[[0.02102704]] [0.]
[[0.02102704]]
[[0.07232016]] [0.]
[[0.07232016]]
[[0.31965549]] [0.]
[[0.31965549]]
[[0.00946333]] [0.]
[[0.00946333]]
[[0.02388417]] [0.]
[[0.02388417]]
[[0.07753809]] [0.]
[[0.07753809]]
[[0.05218481]] [0.]
[[0.05218481]]
[[0.30411559]] [1.]
[[0.30411559]]
[[0.06771614]] [1.]
[[0.06771614]]
[[0.01423513]] [0.]
[[0.01423513]]
[[0.00283074]] [0.]
[[0.00283074]]
[[0.00460776]] [0.]
[[0.00460776]]
[[0.06662802]

[[0.3355213]]
[[0.11024256]] [0.]
[[0.11024256]]
[[0.03538585]] [0.]
[[0.03538585]]
[[0.27251189]] [0.]
[[0.27251189]]
[[0.49233867]] [1.]
[[0.49233867]]
[[0.14535738]] [0.]
[[0.14535738]]
[[0.06868535]] [0.]
[[0.06868535]]
[[0.33288458]] [0.]
[[0.33288458]]
[[0.87198071]] [1.]
[[0.87198071]]
[[0.67930009]] [1.]
[[0.67930009]]
[[0.24576668]] [1.]
[[0.24576668]]
[[0.82903181]] [1.]
[[0.82903181]]
[[0.01596858]] [0.]
[[0.01596858]]
[[0.20208817]] [0.]
[[0.20208817]]
[[0.02445867]] [0.]
[[0.02445867]]
[[0.03994284]] [0.]
[[0.03994284]]
[[0.3749561]] [1.]
[[0.3749561]]
[[0.39003701]] [0.]
[[0.39003701]]
[[0.03625153]] [0.]
[[0.03625153]]
[[0.72779238]] [1.]
[[0.72779238]]
[[0.03058091]] [0.]
[[0.03058091]]
[[0.73115089]] [0.]
[[0.73115089]]
[[0.39229152]] [0.]
[[0.39229152]]
[[0.00044122]] [0.]
[[0.00044122]]
[[0.1608375]] [0.]
[[0.1608375]]
[[0.85272607]] [0.]
[[0.85272607]]
[[0.08155294]] [0.]
[[0.08155294]]
[[0.08103882]] [0.]
[[0.08103882]]
[[0.02247367]] [0.]
[[0.02247367]]
[[0.092778

[[0.00460402]]
[[0.09835493]] [0.]
[[0.09835493]]
[[0.02988902]] [0.]
[[0.02988902]]
[[0.27181869]] [0.]
[[0.27181869]]
[[0.02181875]] [0.]
[[0.02181875]]
[[0.11822114]] [0.]
[[0.11822114]]
[[0.02695107]] [0.]
[[0.02695107]]
[[0.36206303]] [0.]
[[0.36206303]]
[[0.78929059]] [0.]
[[0.78929059]]
[[0.06556904]] [1.]
[[0.06556904]]
[[0.02253596]] [0.]
[[0.02253596]]
[[0.39738253]] [0.]
[[0.39738253]]
[[0.05381481]] [0.]
[[0.05381481]]
[[0.00467735]] [0.]
[[0.00467735]]
[[0.03053007]] [0.]
[[0.03053007]]
[[0.45693987]] [0.]
[[0.45693987]]
[[0.14434213]] [0.]
[[0.14434213]]
[[0.01087768]] [0.]
[[0.01087768]]
[[0.63889888]] [0.]
[[0.63889888]]
[[0.2629371]] [0.]
[[0.2629371]]
[[0.10645911]] [0.]
[[0.10645911]]
[[0.03308518]] [0.]
[[0.03308518]]
[[0.04996801]] [1.]
[[0.04996801]]
[[0.41883688]] [0.]
[[0.41883688]]
[[0.74958493]] [0.]
[[0.74958493]]
[[0.01307811]] [0.]
[[0.01307811]]
[[0.55736459]] [0.]
[[0.55736459]]
[[0.0317196]] [0.]
[[0.0317196]]
[[0.36978881]] [0.]
[[0.36978881]]
[[0.01866

[[0.74373332]]
[[0.11848273]] [0.]
[[0.11848273]]
[[0.03598061]] [0.]
[[0.03598061]]
[[0.37778609]] [1.]
[[0.37778609]]
[[0.03882276]] [0.]
[[0.03882276]]
[[0.37286955]] [1.]
[[0.37286955]]
[[0.03666662]] [0.]
[[0.03666662]]
[[0.17460208]] [0.]
[[0.17460208]]
[[0.0074126]] [0.]
[[0.0074126]]
[[0.10482404]] [0.]
[[0.10482404]]
[[0.0235297]] [0.]
[[0.0235297]]
[[0.35955843]] [0.]
[[0.35955843]]
[[0.03384268]] [0.]
[[0.03384268]]
[[0.50958055]] [0.]
[[0.50958055]]
[[0.22411715]] [1.]
[[0.22411715]]
[[0.27927145]] [1.]
[[0.27927145]]
[[0.00876215]] [0.]
[[0.00876215]]
[[0.14858158]] [0.]
[[0.14858158]]
[[0.00077037]] [0.]
[[0.00077037]]
[[0.05871998]] [0.]
[[0.05871998]]
[[0.00571333]] [0.]
[[0.00571333]]
[[0.24103192]] [0.]
[[0.24103192]]
[[0.30460299]] [0.]
[[0.30460299]]
[[0.09927162]] [0.]
[[0.09927162]]
[[0.00819954]] [0.]
[[0.00819954]]
[[0.84487058]] [1.]
[[0.84487058]]
[[0.41360603]] [1.]
[[0.41360603]]
[[0.0275708]] [0.]
[[0.0275708]]
[[0.23769635]] [1.]
[[0.23769635]]
[[0.0589338

[[0.00423239]]
[[0.02570167]] [0.]
[[0.02570167]]
[[0.01257175]] [0.]
[[0.01257175]]
[[0.01661549]] [0.]
[[0.01661549]]
[[0.02875968]] [1.]
[[0.02875968]]
[[0.26002342]] [0.]
[[0.26002342]]
[[0.45187055]] [0.]
[[0.45187055]]
[[0.36844115]] [0.]
[[0.36844115]]
[[0.0310645]] [0.]
[[0.0310645]]
[[0.335623]] [0.]
[[0.335623]]
[[0.04461496]] [0.]
[[0.04461496]]
[[0.07068613]] [0.]
[[0.07068613]]
[[0.00209554]] [0.]
[[0.00209554]]
[[0.6387195]] [1.]
[[0.6387195]]
[[0.06772013]] [0.]
[[0.06772013]]
[[0.02614834]] [0.]
[[0.02614834]]
[[0.06811171]] [0.]
[[0.06811171]]
[[0.08700325]] [1.]
[[0.08700325]]
[[0.13147564]] [0.]
[[0.13147564]]
[[0.28107697]] [1.]
[[0.28107697]]
[[0.33712363]] [0.]
[[0.33712363]]
[[0.06038556]] [0.]
[[0.06038556]]
[[0.39364163]] [0.]
[[0.39364163]]
[[0.03017893]] [0.]
[[0.03017893]]
[[0.12932847]] [0.]
[[0.12932847]]
[[0.14421836]] [0.]
[[0.14421836]]
[[0.11410745]] [0.]
[[0.11410745]]
[[0.38053064]] [0.]
[[0.38053064]]
[[0.14608042]] [0.]
[[0.14608042]]
[[0.06659819]

In [12]:
print(s)
threshold = 0.65
for i in range(number_of_training_examples):
    print(s[i])
    if s[i] < threshold:
        s[i] = 0
    else:
        s[i] = 1
    print(s[i])

[0.02525975 0.02287932 0.04438919 ... 0.0445903  0.33971589 0.1226301 ]
0.025259748064295214
0.0
0.022879318130369284
0.0
0.04438918525438696
0.0
0.3460302815195597
0.0
0.030763767921990218
0.0
0.22339999266457622
0.0
0.057937992969349796
0.0
0.03640015861695946
0.0
0.6603761181881224
1.0
0.049825198751092856
0.0
0.8150590962383305
1.0
0.0566495835800572
0.0
0.01885271317686973
0.0
0.059598795733114544
0.0
0.14170398411297128
0.0
0.0075104300669149
0.0
0.09585021246205411
0.0
0.9157054633363154
1.0
0.25715216281853326
0.0
0.4127591037975263
0.0
0.2805540012898408
0.0
0.39235715696553697
0.0
0.2433591301801831
0.0
0.021807092893692683
0.0
0.005172685014002987
0.0
0.0031032208659996586
0.0
0.00928448307976435
0.0
0.07860435644428082
0.0
0.015837658828971705
0.0
0.1794717077920241
0.0
0.04756140419853729
0.0
0.07168902010209112
0.0
0.012989639399508876
0.0
0.011173003298500668
0.0
0.07561639502288058
0.0
0.020249947009433486
0.0
0.015019818619283223
0.0
0.228114247873794
0.0
0.82227057603

0.027525776811470316
0.0
0.029185614196749483
0.0
0.015381792048294658
0.0
0.01803576253194181
0.0
0.07500477539911272
0.0
0.027585590262284406
0.0
0.3898977562732113
0.0
0.046547760698885256
0.0
0.005516684070506834
0.0
0.08293223514974507
0.0
0.9224457567982944
1.0
0.38100016515019397
0.0
0.0034472221063168457
0.0
0.0033888610971252527
0.0
0.0095434202921146
0.0
0.12884306068157475
0.0
0.09159854847755554
0.0
0.003080145727112769
0.0
0.06450914941812343
0.0
0.07196456396299042
0.0
0.06422603792402283
0.0
0.09205085157006306
0.0
0.2839642658893999
0.0
0.20485960499027184
0.0
0.031393993325137204
0.0
0.018859117984373536
0.0
0.7987084530257332
1.0
0.004422247532655197
0.0
0.013872935434135045
0.0
0.018500426736344226
0.0
0.08247238185468998
0.0
0.03765126379654565
0.0
0.09316315369755404
0.0
0.025751980014758744
0.0
0.06142377137929607
0.0
0.014163717068709588
0.0
0.31511446532385223
0.0
0.009135060223384483
0.0
0.14488934003232978
0.0
0.009045557025475437
0.0
0.8860538129628396
1.0
0.

0.0
0.005328787259400918
0.0
0.01743164332667122
0.0
0.35917596457751727
0.0
0.04124339872924297
0.0
0.33439028349826366
0.0
0.0508294153966933
0.0
0.9027599375022641
1.0
0.03451875631691524
0.0
0.09729237934040426
0.0
0.08331160087112666
0.0
0.05346007592975025
0.0
0.9119727039035579
1.0
0.040282147149709985
0.0
0.013284270644784758
0.0
0.0027311454160688013
0.0
0.09167308474943378
0.0
0.05294602370787066
0.0
0.1928446997624483
0.0
0.09347019832610788
0.0
0.4625310661072975
0.0
0.09751325487756148
0.0
0.007463628163178384
0.0
0.025348455565534625
0.0
0.3864797306353484
0.0
0.7150799496065016
1.0
0.006860093066534305
0.0
0.07487585003333948
0.0
0.02522009848467218
0.0
0.03820199649562753
0.0
0.011300963736234942
0.0
0.1621349313817487
0.0
0.09473861486099072
0.0
0.3355212954236852
0.0
0.11024256389202329
0.0
0.03538585248350878
0.0
0.2725118910790741
0.0
0.49233866989323355
0.0
0.1453573778549717
0.0
0.06868535431653688
0.0
0.3328845827504312
0.0
0.8719807142963346
1.0
0.67930008737620

1.0
0.026573451023290857
0.0
0.15151089234605833
0.0
0.9215439880325902
1.0
0.33954077247715436
0.0
0.3570433924928385
0.0
0.06335174863925623
0.0
0.012884090118107738
0.0
0.020438106618067053
0.0
0.0030531342246686295
0.0
0.062021831046850454
0.0
0.03906787973727338
0.0
0.36574672802502844
0.0
0.08180570813305507
0.0
0.0962445946990023
0.0
0.8715173699020183
1.0
0.14915929055115795
0.0
0.18481888147199677
0.0
0.01806411523045794
0.0
0.012020966415206311
0.0
0.0571754400348838
0.0
0.014515673563594641
0.0
0.021300182082589377
0.0
0.07359693780413913
0.0
0.027650078204117582
0.0
0.5935961360399479
0.0
0.005008591986418854
0.0
0.8779897685010177
1.0
0.026366499892822522
0.0
0.2558378495134482
0.0
0.1782783066993342
0.0
0.04288421571237605
0.0
0.751020197055169
1.0
0.03687741127338252
0.0
0.12638594478989787
0.0
0.004423029448527008
0.0
0.07289517964475592
0.0
0.03365678322474281
0.0
0.0030406743603493026
0.0
0.35337670541052035
0.0
0.22453211981448531
0.0
0.007220448105296064
0.0
0.00721

0.09484558823495295
0.0
0.02698498553735839
0.0
0.19097721798310488
0.0
0.2258896173522651
0.0
0.025844389303734217
0.0
0.23640932187538177
0.0
0.747449730653448
1.0
0.0902187959803384
0.0
0.008626655558217986
0.0
0.5143182451493893
0.0
0.1827018719573224
0.0
0.044117595556407464
0.0
0.009924278239952474
0.0
0.7546114529627701
1.0
0.25787544830358816
0.0
0.022605493104738437
0.0
0.08585502690904971
0.0
0.12527459665461063
0.0
0.17449933534410056
0.0
0.02175797262638047
0.0
0.08136023428936362
0.0
0.005417996517152825
0.0
0.5279841556398746
0.0
0.5890260622744452
0.0
0.003575824515151501
0.0
0.05046241829694055
0.0
0.0528341951577321
0.0
0.15942328760552815
0.0
0.07913825267292635
0.0
0.010569661649660732
0.0
0.01651207645267302
0.0
0.39584204298694403
0.0
0.004232388295443328
0.0
0.025701673257660214
0.0
0.012571745495148856
0.0
0.016615492272191123
0.0
0.02875967737096344
0.0
0.26002342267898126
0.0
0.45187054609644495
0.0
0.3684411508061481
0.0
0.031064502853144264
0.0
0.335622996743

In [13]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, s))
for i in range(number_of_training_examples):
    print(np.c_[s, y_test][i])

0.852
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[1. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[1. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[1. 0.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 1.]
[0. 0.]
[0. 1.]
[0. 0.]
[1. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 1.]
[0. 0.]
[1. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[1. 1.]
[1. 1.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 1.]
[1. 1.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[1. 1.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 1.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0

[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[1. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[1. 1.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]
[1. 1.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[1. 1.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 0.]
[0. 1.]


In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, s)
cm

array([[1563,   47],
       [ 249,  141]], dtype=int64)

In [17]:
accuracy = (cm[0, 0] + cm[1, 1]) / (cm[0, 0] + cm[1, 0] + cm[0, 1] + cm[1, 1])
print(accuracy)

0.852


In [ ]:
l